In [25]:
from langchain.agents import load_tools, Tool
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
import os
from langchain import SQLDatabase, SQLDatabaseChain
import yaml

from langchain.agents import create_openapi_agent
from langchain.agents.agent_toolkits import OpenAPIToolkit
from langchain.llms.openai import OpenAI
from langchain.requests import RequestsWrapper
from langchain.tools.json.tool import JsonSpec
from langchain.chat_models.openai import ChatOpenAI
from langchain.agents.chat.base import ChatAgent
from langchain.agents import AgentExecutor

os.environ["OPENAI_API_KEY"] = ""

llm = OpenAI(temperature=0)

db = SQLDatabase.from_uri("""postgresql://postgres:TnZe*sJY3OMe|"%<@34.148.191.128""")

db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

with open("nutritionix.yml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
json_spec=JsonSpec(dict_=data, max_value_length=4000)
headers = {
    "x-app-id": "13509888",
    "x-app-key": "dbacf5040facc8cbfa01ebc89cf3ed60",
    "x-remote-user-id": 0
}
requests_wrapper=RequestsWrapper(headers=headers)
openapi_toolkit = OpenAPIToolkit.from_llm(OpenAI(temperature=0), json_spec, requests_wrapper, verbose=True)
openapi_agent_executor = create_openapi_agent(
    llm=OpenAI(temperature=0),
    toolkit=openapi_toolkit,
    verbose=True
)

tools = [
    Tool(
            name = "SQL Database",
            func=db_chain.run,
            description="""Runs a query on an SQL Database. The following tables are in the db:
            
            users:
                -id: uuid
                -name: text
                -email: text
                -age:int4
                -weight: float4
                -height: text
                -body fat: float4
                -token: text

                goals:
                -id: uuid
                -user_id: uuid
                -goal weight: float4
                -goal body fat: float4

                workout_split:
                -id: uuid
                -user_id: uuid
                -description: text

                workouts:
                -id: uuid
                -user_id: uuid
                -summary: text
                -exercise: text
                -calories burned: int4
                -time: timestamp

                meals:
                -id: uuid
                -user_id: uuid
                -meal name: text
                -ingredients: text
                -calories: int4
                -fat: int4
                -protein: int4
                -carbs: int4
                -meal type: text
                -time: timestamp

                PRs:
                -id: uuid
                -user_id: uuid
                -exercise: text
                -weight: float4
                -time: timestamp
            """
    ),
    Tool(
        name = "Nutrition Information",
        func = openapi_agent_executor.run,
        description = "Can retrieve nutritional information from a natural language query."
    )
]

agent = ChatAgent.from_chat_model_and_tools(ChatOpenAI(temperature=0), tools)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

agent_executor.run("Get the nutritional value of an egg and then log it into the database as a meal eaten by 9d20bad4-6856-4679-85ee-340cc8c96169")



> Entering new AgentExecutor chain...


KeyboardInterrupt: 